In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import moviepy
from skimage.feature import hog
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import os
import pandas as pd
import random
from scipy.ndimage.measurements import label
from moviepy.editor import VideoFileClip
import pickle

%matplotlib inline

In [3]:
## FUNCTIONS FROM LECTURE ##


def hog_features(img, orient, pix_per_cell, cell_per_block,
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient,
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block),
                                  transform_sqrt=False,
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:
        features = hog(img, orientations=orient,
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block),
                       transform_sqrt=False,
                       visualise=vis, feature_vector=feature_vec)
        return features

def color_hist(img, nbins=32):    #bins_range=(0, 256)
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

### Step 1: Train a classifier 
(using HOG features with appended color hist features)

In [4]:
# sharpens the edges of image
def sharpen_img(img):
    kernel = np.array([[-1,-1,-1,-1,-1],
                         [-1,2,2,2,-1],
                         [-1,2,16,2,-1],
                         [-1,2,2,2,-1],
                         [-1,-1,-1,-1,-1]]) / 16.
    sharp_img = cv2.filter2D(img, -1, kernel)
    return sharp_img

In [5]:
# get the image locations
def get_image_locs():
    # save car image locations
    cars_main_folder = './data/vehicles_smallset/'
    cars = [] # store file paths of car images here
    cars_subdirs = [sub for sub,_,_ in os.walk(cars_main_folder)][1:]# first entry is that folder itself
    # iterate through the contents of subfolders to get image paths
    for cars_subdir in cars_subdirs:
        for file in os.listdir(cars_subdir):
            if '.jpeg' not in file: continue # all images here are jpeg files
            file_path = cars_subdir + '/' + file
            cars.append(file_path)
    # save non-car image locations
    non_cars_main_folder = './data/non-vehicles_smallset/'
    non_cars = [] # store file paths of car images here
    non_cars_subdirs = [sub for sub,_,_ in os.walk(non_cars_main_folder)][1:]
    # iterate through the contents of subfolders to get image paths
    for non_cars_subdir in non_cars_subdirs:
        for file in os.listdir(non_cars_subdir):
            if '.jpeg' not in file: continue # all images here are jpeg 
            file_path = non_cars_subdir + '/' + file
            non_cars.append(file_path)
    
    return cars, non_cars

In [6]:
# get images in gray scale or color (RGB)
def get_images(img_path_lst, color_channel='gray', flat=False, return_image_info=False): 
    img_lst = []
    for img_path in img_path_lst:
        img = plt.imread(img_path)
        if return_image_info: # currently just the shape
            img_shape = img.shape
        if color_channel == 'gray':
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        if color_channel == 'HLS':
            img = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        if color_channel == 'YCrCb':
            img = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
        if color_channel == 'LUV':
            img = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        if flat:
            img = img.flatten()
        img_lst.append(img)
    if return_image_info: return img_lst, img_shape
    return img_lst

In [7]:
# returns a list of arrays of HOG features
# no grayscale
def get_hog_feats(img_lst, sharpen=True, orient=9, pix_per_cell=8, cell_per_block=3):
    
    img_hog_lst = []
    for img in img_lst:
        if sharpen: img = sharpen_img(img)
        ch1,ch2,ch3 = img[:,:,0], img[:,:,1], img[:,:,2]
        all_ch_hog = []
        for ch in [ch1,ch2,ch3]:
            ch_hog = hog_features(ch, orient, pix_per_cell, cell_per_block,
                            vis=False, feature_vec=True)
            all_ch_hog.append(ch_hog)
        img_hog = np.concatenate(all_ch_hog)
        img_hog_lst.append(img_hog)
        
    return img_hog_lst

In [8]:
# returns a list of arrays of the color histograms
def add_color_hist(img_lst, bins=32):
    
    img_color_hist_lst = []
    for img in img_lst:
        img_color_hist = color_hist(img, nbins=bins)
        img_color_hist_lst.append(img_color_hist)
    
    return img_color_hist_lst    

In [9]:
# combines two feature arrays, min max scales, and (optionally) returns the scaler object for use later
def combine_scale_split(X_1, X_2, y, scale=True, return_scaler=False):
    
    X_1 = np.array(X_1)
    X_2 = np.array(X_2)
    
    X = np.hstack((X_1,X_2)) # cbind
    
    if scale:
        scaler = RobustScaler().fit(X)
        X_scaled = scaler.transform(X)
    else:
        X_scaled = X
        
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=99)
    
    if return_scaler:
        return X_train, X_test, y_train, y_test, scaler
    return X_train, X_test, y_train, y_test

In [10]:
# machine learning pipeline (uses SVM classifier)

def classification_pipeline(car_path_lst, non_car_path_lst, color_map, bins, HOG_sharpen, 
                            HOG_orient, HOG_pix_per_cell, HOG_cell_per_block, svm_kernel):
    
    #global X_1, X_2 # added for debugging
    car_lst = get_images(car_path_lst, color_channel=color_map); car_len = len(car_lst)
    non_car_lst = get_images(non_car_path_lst, color_channel=color_map); non_car_len = len(non_car_lst)
    complete_lst = car_lst + non_car_lst
    y = ['car']*car_len + ['not_car']*non_car_len # this is the target array
    # get the hog features
    X_1 = get_hog_feats(complete_lst, sharpen=HOG_sharpen, orient=HOG_orient, 
                       pix_per_cell=HOG_pix_per_cell, cell_per_block=HOG_cell_per_block)
    
    # get color histograms
    X_2 = add_color_hist(complete_lst, bins=bins)
    
    # combine, scale, and get the test and train sets
    X_train, X_test, y_train, y_test, scaler = combine_scale_split(X_1, X_2, y, scale=True, return_scaler=True)
    
    # apply SVC to get performance
    fit = SVC(kernel=svm_kernel).fit(X_train, y_train)
    score = fit.score(X_test, y_test)
    
    return score, fit, scaler 


In [11]:
# get the performance for different parameters
def grid_search(car_paths, non_car_paths, 
                params ={
                  'color_map_lst':['HLS','YCrCb','LUV'],
                  'bins':[32,64],
                  'sharpen':[True],
                  'orient':[9,11],
                  'pix_per_cell':[8],
                  'cell_per_block':[2],
                  'svm_kernel':['rbf','linear']
                 }):
    
    df = pd.DataFrame(columns=['Color_map','Bins','HOG_sharpen','HOG_orientation',
                               'HOG_pixels_cell','HOG_cells_block', 'SVM_kernel','SCORE'])
    best_score = 0
    i = 0 
    for color in params['color_map_lst']:
        for bin_ in params['bins']:
            for sharp in params['sharpen']:
                for orientation in params['orient']:
                    for pix in params['pix_per_cell']:
                        for cell in params['cell_per_block']:
                            for kernel in params['svm_kernel']:
                                
                                score, fit,_ = classification_pipeline(car_paths, non_car_paths,
                                                                            color,bin_,sharp,orientation,pix,
                                                                             cell,kernel)
                                params_lst = [color, bin_, sharp, orientation, pix, cell, kernel, score]
                                i += 1 # increment to next index
                                df.loc[i] = params_lst # add row to dataframe
                                print('Combination: '+str(i)+' done. Score: ' + str(score))
                                if score >= best_score: # check if score has improved
                                    best_fit = fit
                                    #best_scaler = scaler 
                                    best_params = params_lst
                                    best_score = score
    
    #return best_fit, best_scaler, best_score, best_params, df
    return best_fit, best_score, best_params, df
    

#### Perform a Grid Search to find best param combination
We have 24 combinations

Hence, based on this search we are getting a very high test set accuracy of 0.989 using YCrCb color map with a combination of color histogram with 64 bins and HOG features with 9 different orientations, 8 pixels per cell, and 2 cells per block. We also sharpened the image and a linear kernel for SVM.

The *fit_* object can be used be used to classify an unknown image and the *scaler_* object is used to scale data using the same values that were applied to the data used in the chosen model. This will ensure accuracte results.

Since HOG is **not** scale invariant| we need to first resize test image before classifying it. 

#### Retrain the classifier using the chosen parameters on full data

In [21]:
# save car image locations
cars_main_folder = './full_data/vehicles/'
cars = [] # store file paths of car images here
cars_subdirs = [sub for sub,_,_ in os.walk(cars_main_folder)][1:]# first entry is that folder itself
# iterate through the contents of subfolders to get image paths
for cars_subdir in cars_subdirs:
    for file in os.listdir(cars_subdir):
        if '.jpeg' in file or '.png' in file or '.jpg' in file: 
            file_path = cars_subdir + '/' + file
            cars.append(file_path)
# save non-car image locations
non_cars_main_folder = './full_data/non-vehicles/'
non_cars = [] # store file paths of car images here
non_cars_subdirs = [sub for sub,_,_ in os.walk(non_cars_main_folder)][1:]
# iterate through the contents of subfolders to get image paths
for non_cars_subdir in non_cars_subdirs:
    for file in os.listdir(non_cars_subdir):
        if '.jpeg' in file or '.png' in file or '.jpg' in file:  
            file_path = non_cars_subdir + '/' + file
            non_cars.append(file_path)
    

In [23]:
len(non_cars)

8968

In [ ]:
# Retrain classifier using above parameters
car_lst = get_images(cars, color_channel='YCrCb'); car_len = len(car_lst)
non_car_lst = get_images(non_cars, color_channel='YCrCb'); non_car_len = len(non_car_lst)
complete_lst = car_lst + non_car_lst
y = ['car']*car_len + ['not_car']*non_car_len # this is the target array

# get hog features
X_1 = get_hog_feats(complete_lst, sharpen=True, orient=11, 
                   pix_per_cell=8, cell_per_block=2)

# get color histograms
complete_lst = car_lst + non_car_lst
X_2 = add_color_hist(complete_lst, bins=32)

# combine, scale, and get the test and train sets
X_train, X_test, y_train, y_test, scaler = combine_scale_split(X_1, X_2, y, scale=True, return_scaler=True)

# apply SVC to get performance
fit = SVC(kernel='linear',probability=True).fit(X_train, y_train)
score = fit.score(X_test, y_test)

In [ ]:
# final classifier score:
print(score)
# save model and scaler for future use without retraining
import pickle
model = {'fit':fit,'scaler':scaler}
with open('./model.p','wb') as f:
    pickle.dump(model,f)

In [12]:
############# IF RESTARTING, GET MODEL AND SCALER FROM HERE ##################
loader = pickle.load(open('./model.p','rb'))
fit, scaler = loader['fit'], loader['scaler']

In [13]:
def classify_img(img, fit=fit, scaler=scaler): # input img must be rgb
    
    # resize image to the same size as images in training data
    img = cv2.resize(img,(64,64))
    # sharpen image
    img = sharpen_img(img)
    # get images with right color maps
    color = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    # get features
    #global hog_feats, color_hist_feats
    # hog
    ch_hog_lst = []
    for ch in [color[:,:,0], color[:,:,1], color[:,:,2]]:
        ch_hog = hog_features(ch,orient=11, pix_per_cell=8, cell_per_block=2)
        ch_hog_lst.append(ch_hog)
    hog_feats = np.concatenate(ch_hog_lst)
    # color hist
    color_hist_feats = color_hist(color, nbins=32)
    all_feats = np.hstack([hog_feats, color_hist_feats])
    # scaling image
    scaled = scaler.transform([all_feats])
    # predict
    prediction = fit.predict(scaled)
    proba = fit.predict_proba(scaled)
    
    return prediction[0], proba[0][0] # the first probability result corresponds to car

### Step 2: Implement Sliding Window of different window sizes

In [14]:
# takes in an image and returns coordinates of windows with positive car detections
# the quickest approach is to avoid overlapping windows of the same size
# vis=True draws all rectangles (not just positive ones)
# each vertical area will be covered by a different window 
def find_good_windows(img, start_hw=[400,400], h_bounds=[475,550,620], w_bounds=[720,750,850],
                      window_sizes=[65,85,120], factor=[0.2,0.3,0.15], w_max=[1000,1200,1300], viz=False):
    
    if viz: img_copy = np.copy(img)
    positive_windows = [] # list to hold all windows that contain a car
    start_h, start_w = start_hw
    #sizes_count = max_size//min_size # number of different window sizes
    #curr_size = min_size
    
    for curr_size, end_h, start_w_wind, factor, end_w  in zip(window_sizes, 
                                                       h_bounds, w_bounds, factor, w_max):
        delta_h, delta_w = curr_size, curr_size
        start_w = start_w_wind
        # pick a different color for each window
        if viz: color = (random.randint(190,255), random.randint(190,255), random.randint(190,255))
        while True:
            # if we reach right corner then break
            if start_w + delta_w > end_w and start_h + delta_h >= end_h: 
                start_h, start_w = start_hw # go to starting point of width
                break
            # if we traversed the entire width but not the height
            if start_w + delta_w > end_w and start_h + delta_h < end_h:
                start_h += int(delta_h*factor)
                start_w = start_w_wind # go to starting point of width
            # if not crossing image border
            image_window = img[start_h:start_h+delta_h, start_w:start_w+delta_w,:]
            if viz: cv2.rectangle(img_copy, (start_w, start_h), (start_w+delta_w, start_h+delta_h), color, 6)
            pred, prob = classify_img(image_window)
            if prob > 0.55: # higher probability used to weed out some of the false positives
                # if positive (car detected, append the diagnol coords of bounding box)
                # since for some odd reason cv2 uses y,x not x,y (w,h not h,w)
                positive_windows.append([(start_w, start_h),(start_w+delta_w, start_h+delta_h)])
                #print(delta_h, delta_w)
                # color a positive window in red
                if viz: cv2.rectangle(img_copy, (start_w, start_h), (start_w+delta_w, start_h+delta_h), (255,0,0), 6)
                
            start_w += int(delta_w*factor)
            #start_h = end_h
    
    if viz:
        plt.imshow(img_copy)
        plt.title('Image with all windows')
        plt.show()
    
    # return only the windows that have a positive car detection
    return positive_windows

In [15]:
def draw_boxes(img, bbox_lst):
    img_copy = np.copy(img)
    for bbox in bbox_lst:
        cv2.rectangle(img_copy, bbox[0], bbox[1], color=(0,0,255), thickness=6)
    return img_copy

### Step 3: Create a heatmap of recurring detections frame by frame and follow detected vehicles using centroid of detection areas


In [16]:
def gen_heat_map(img, good_boxes, viz=False):
    heat_map = np.zeros_like(img[:,:,0]).astype(np.float32)
    for box in good_boxes:
        # box is (y1,x1),(y2,x2) -- y,x format chosen for opencv
        (y1,x1),(y2,x2) = box
        heat_map[x1:x2,y1:y2] += 1
    if viz:
        plt.imshow(heat_map)
        plt.title('heat map')
    return heat_map
    

In [17]:
# applies a threshold to the heat map
# this helps weed out false postives
def apply_thresh(heat_map, thresh='auto', auto_factor=1, viz=False):
    if thresh == 'auto': # if auto, uses the average intensity of only cells with detections
        # max() used to account for images with NO detections (div by 0 error prevention)
        thresh = np.sum(heat_map)/max(np.sum(heat_map > 0),1) * auto_factor
        print('threshold of ', str(thresh))
    
    bin_map = heat_map >= thresh
    
    if viz: 
        plt.imshow(bin_map)
        plt.title('heat map after thresholding')
    return bin_map
    

### Step 4: Estimate a bounding box for detected vehicles



In [18]:
# takes in the binary image without the thresholds and estimates the bouding box for each vechicle
def get_bounding_boxes(img, bin_map, viz=False):
    # copy img
    img_copy = np.copy(img)
    # get unique regions
    labeled_img, region_count = label(bin_map)
    # viz
    if viz: plt.imshow(labeled_img); plt.title('Image with regions')
    # for each region_id create a bbox based on max and min x
    for region_id in range(1,region_count+1):
        # get the coords
        region_coords = np.nonzero(labeled_img == region_id)
        h,w = region_coords # h or x is vertical, w or y is horizontal
        # gen the coordinates of bbox (y,x) or (w,h) is opencv format
        top_left = (np.min(w), np.min(h)) # min h is higher than max h
        bottom_right = (np.max(w), np.max(h))
        if top_left == (0,0): continue # indicates there is no detection
        # another false positive removal step - assumes that small bounding boxes are false positives
        diagnol = np.sqrt((top_left[0] - bottom_right[0])**2 + (top_left[1] - bottom_right[1])**2) 
        print(diagnol)
        if bottom_right[1] > 600: 
            if diagnol <= 100: continue
        # draw the rectangle
        cv2.rectangle(img_copy, top_left, bottom_right, (0,255,0), 4)
        
    if viz: plt.imshow(img_copy); plt.title('Final Image with Vehicles Detected')
    
    return img_copy

### Step 5: Pipeline to put it all together and run on video

In [19]:
save_errors = [] # save errors to look later

def vehicle_detection_pipeline(img):
    
    try:
        img_copy = np.copy(img)
        # sliding windows on image
        good_windows = find_good_windows(img_copy, viz=False)
        # generate a heatmap
        heat = gen_heat_map(img_copy, good_windows, viz=False)
        # apply a threshold to remove false positives
        thresh_img = apply_thresh(heat, thresh=1, viz=False)
        # generate final image with bouding boxes around vehicles
        final_img = get_bounding_boxes(img, thresh_img, viz=False)
        
    except Exception as e:
        print(e)
        save_errors.append(e)
    
    return final_img

#### Run pipeline on video

In [20]:
#### APPLY PIPELINE TO VIDEO ####
output_video_file = './results/project_video_out.mp4'
input_video = VideoFileClip('./project_video.mp4')
#input_video = VideoFileClip('./project_video.mp4').subclip(20,30)
#input_video = VideoFileClip('./test_video.mp4')
output_video = input_video.fl_image(vehicle_detection_pipeline)
%time output_video.write_videofile(output_video_file, audio=False, fps=15)

[MoviePy] >>>> Building video ./results/project_video_out.mp4
[MoviePy] Writing video ./results/project_video_out.mp4


 12%|█████████▉                                                                       | 93/757 [05:57<42:51,  3.87s/it]

158.700976683


 12%|██████████                                                                       | 94/757 [06:01<42:46,  3.87s/it]

197.87369709


...

137.611772752


100%|███████████████████████████████████████████████████████████████████████████████▊| 755/757 [48:47<00:07,  3.87s/it]

137.611772752


100%|███████████████████████████████████████████████████████████████████████████████▉| 756/757 [48:51<00:03,  3.87s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./results/project_video_out.mp4 

Wall time: 48min 52s


In [ ]:
save_errors